In [66]:
import findspark
findspark.init('/home/sdmohant/spark-3.0.0-bin-hadoop2.7')

import pyspark
import random

In [67]:
from pyspark.sql import SparkSession

In [68]:
from pyspark import SparkContext
#from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *


In [69]:
#sc = SparkContext('local')

In [70]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [71]:
sc = spark.sparkContext

In [72]:
txt_file = sc.textFile("../Data/graph/2018-02-02/ConferenceSeries.txt")

In [73]:
sqlContext = SQLContext(sc)

In [74]:
header = txt_file.first()

In [75]:
fields = [StructField(field_name, StringType(),True) for field_name in header.split("\\t") ]

In [76]:
schema = StructType(fields)

In [77]:
txt_file = txt_file.filter(lambda line : line!= header)

In [78]:
txt_file.take(10)

[u'1141689323\t11674\tIEEM\tIndustrial Engineering and Engineering Management\t7218\t7851',
 u'2727030889\t13563\tBigDataSecurity\tInternational Conference on Big Data Security on Cloud \t120\t122',
 u'2757782454\t25000\tSSVM\tScale Space and Variational Methods in Computer Vision\t1\t21',
 u'2754909024\t13978\tICARM\tInternational Conference on Advanced Robotics and Mechatronics\t115\t31',
 u'2758502524\t11494\tGreenCom\tGreen Computing and Communications\t417\t1947',
 u'2755154462\t11589\tBodyNets\tInternational Conference on Body Area Networks\t475\t2605',
 u'2755191446\t12332\tIWDDC\tInternational Waveform Diversity and Design Conference\t291\t1457',
 u'2755241531\t10609\tFG \tIEEE International Conference on Automatic Face & Gesture Recognition\t736\t11601',
 u'2755283090\t10519\tICAI\tInternational Conference on Artificial Intelligence\t4485\t10952',
 u'1171878286\t12080\tVaMoS\tVariability Modelling of Software-Intensive Systems\t247\t2633']

In [79]:
temp_var = txt_file.map(lambda k: k.split("\t"))

In [80]:
df = temp_var.toDF()

In [81]:
df.show()

+----------+-----+---------------+--------------------+----+------+
|        _1|   _2|             _3|                  _4|  _5|    _6|
+----------+-----+---------------+--------------------+----+------+
|1141689323|11674|           IEEM|Industrial Engine...|7218|  7851|
|2727030889|13563|BigDataSecurity|International Con...| 120|   122|
|2757782454|25000|           SSVM|Scale Space and V...|   1|    21|
|2754909024|13978|          ICARM|International Con...| 115|    31|
|2758502524|11494|       GreenCom|Green Computing a...| 417|  1947|
|2755154462|11589|       BodyNets|International Con...| 475|  2605|
|2755191446|12332|          IWDDC|International Wav...| 291|  1457|
|2755241531|10609|            FG |IEEE Internationa...| 736| 11601|
|2755283090|10519|           ICAI|International Con...|4485| 10952|
|1171878286|12080|          VaMoS|Variability Model...| 247|  2633|
|2755858460|13282|           RSMN|IEEE Regional Sym...| 374|   374|
|2622835322|10826|          ICUWB|International 

In [82]:
df.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)



In [83]:
df = df.withColumn("ConfSeriesID",df["_1"].cast(LongType()))

In [84]:
df = df.withColumn("CSRank",df["_2"].cast(LongType()))

In [85]:
df = df.withColumn("CSNormalizedName",df["_3"].cast(StringType()))

In [86]:
df = df.withColumn("CSDisplayName",df["_4"].cast(StringType()))

In [87]:
df = df.withColumn("CSPaperCount",df["_5"].cast(IntegerType()))

In [88]:
df = df.withColumn("CSCitation",df["_6"].cast(IntegerType()))

In [89]:
df.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)
 |-- ConfSeriesID: long (nullable = true)
 |-- CSRank: long (nullable = true)
 |-- CSNormalizedName: string (nullable = true)
 |-- CSDisplayName: string (nullable = true)
 |-- CSPaperCount: integer (nullable = true)
 |-- CSCitation: integer (nullable = true)



In [90]:
df.show(10)

+----------+-----+---------------+--------------------+----+-----+------------+------+----------------+--------------------+------------+----------+
|        _1|   _2|             _3|                  _4|  _5|   _6|ConfSeriesID|CSRank|CSNormalizedName|       CSDisplayName|CSPaperCount|CSCitation|
+----------+-----+---------------+--------------------+----+-----+------------+------+----------------+--------------------+------------+----------+
|1141689323|11674|           IEEM|Industrial Engine...|7218| 7851|  1141689323| 11674|            IEEM|Industrial Engine...|        7218|      7851|
|2727030889|13563|BigDataSecurity|International Con...| 120|  122|  2727030889| 13563| BigDataSecurity|International Con...|         120|       122|
|2757782454|25000|           SSVM|Scale Space and V...|   1|   21|  2757782454| 25000|            SSVM|Scale Space and V...|           1|        21|
|2754909024|13978|          ICARM|International Con...| 115|   31|  2754909024| 13978|           ICARM|Int

In [91]:
drop_columns =["_1","_2","_3","_4","_5","_6"]
df = df.drop(*drop_columns)               

In [92]:
df.show()

+------------+------+----------------+--------------------+------------+----------+
|ConfSeriesID|CSRank|CSNormalizedName|       CSDisplayName|CSPaperCount|CSCitation|
+------------+------+----------------+--------------------+------------+----------+
|  1141689323| 11674|            IEEM|Industrial Engine...|        7218|      7851|
|  2727030889| 13563| BigDataSecurity|International Con...|         120|       122|
|  2757782454| 25000|            SSVM|Scale Space and V...|           1|        21|
|  2754909024| 13978|           ICARM|International Con...|         115|        31|
|  2758502524| 11494|        GreenCom|Green Computing a...|         417|      1947|
|  2755154462| 11589|        BodyNets|International Con...|         475|      2605|
|  2755191446| 12332|           IWDDC|International Wav...|         291|      1457|
|  2755241531| 10609|             FG |IEEE Internationa...|         736|     11601|
|  2755283090| 10519|            ICAI|International Con...|        4485|    

In [93]:
df.select("CSDisplayName").collect()

[Row(CSDisplayName=u'Industrial Engineering and Engineering Management'),
 Row(CSDisplayName=u'International Conference on Big Data Security on Cloud '),
 Row(CSDisplayName=u'Scale Space and Variational Methods in Computer Vision'),
 Row(CSDisplayName=u'International Conference on Advanced Robotics and Mechatronics'),
 Row(CSDisplayName=u'Green Computing and Communications'),
 Row(CSDisplayName=u'International Conference on Body Area Networks'),
 Row(CSDisplayName=u'International Waveform Diversity and Design Conference'),
 Row(CSDisplayName=u'IEEE International Conference on Automatic Face & Gesture Recognition'),
 Row(CSDisplayName=u'International Conference on Artificial Intelligence'),
 Row(CSDisplayName=u'Variability Modelling of Software-Intensive Systems'),
 Row(CSDisplayName=u'IEEE Regional Symposium on Micro and Nanoelectronics'),
 Row(CSDisplayName=u'International Conference on Ultra-Wideband'),
 Row(CSDisplayName=u'Artificial Intelligence and Pattern Recognition'),
 Row(CSDi

In [94]:
df.select("CSNormalizedName").distinct().count()

3741

In [95]:
df.describe().show()

+-------+-------------------+------------------+----------------+--------------------+------------------+------------------+
|summary|       ConfSeriesID|            CSRank|CSNormalizedName|       CSDisplayName|      CSPaperCount|        CSCitation|
+-------+-------------------+------------------+----------------+--------------------+------------------+------------------+
|  count|               4025|              4025|            4025|                4025|              4025|              4025|
|   mean|2.231268664235528E9|12096.311801242236|            null|                null|1009.2136645962732| 7998.033788819876|
| stddev|7.289105282795117E8|1440.8539098345286|            null|                null|2542.1383528697484|31757.625927609926|
|    min|         1119369170|              6941|             3CA|3DTV-Conference: ...|                 1|                 0|
|    max|         2760805818|             25000|     sensorcomm |workshop on Middl...|             45543|            645856|


In [96]:
from pyspark.sql.functions import isnan,when,count,col

In [97]:
df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+------------+------+----------------+-------------+------------+----------+
|ConfSeriesID|CSRank|CSNormalizedName|CSDisplayName|CSPaperCount|CSCitation|
+------------+------+----------------+-------------+------------+----------+
|           0|     0|               0|            0|           0|         0|
+------------+------+----------------+-------------+------------+----------+



In [98]:
df.coalesce(1).write.format('com.databricks.spark.csv').mode('overwrite').save('ConferenceSeries.csv',header = 'true')

# Conference Instances

In [35]:
txt_file_ci = sc.textFile("../Data/graph/2018-02-02/ConferenceInstances.txt")

In [36]:
txt_file_ci.take(5)

[u'2312404054\t25000\tsiggraph 2016\tSIGGRAPH 2016\t1164321581\tAnaheim, California\thttp://s2016.siggraph.org/call-submissions\t2016-07-24\t2016-07-28\t\t2016-01-18\t\t2016-01-19\t846\t1741',
 u'121793801\t25000\twsdm 2012\tWSDM 2012\t1120384002\tSeattle, WA\thttp://wsdm2012.org/\t2012-02-08\t2012-02-12\t2011-08-04\t2011-08-11\t2011-10-26\t\t87\t3496',
 u'2625789444\t25000\tiprm 2006\tIPRM 2006\t2621445615\tPrinceton, NJ, USA\t\t\t\t\t\t\t\t\t',
 u'82574969\t25000\tfoga 2009\tFOGA 2009\t1159491267\tOrlando, FL, USA\thttp://www.sigevo.org/foga-2009\t2009-01-09\t2009-01-11\t\t2008-09-01\t\t\t19\t382',
 u'196500321\t25000\tkmis 2013\tKMIS 2013\t1158252963\tVilamoura, Algarve, Portugal\thttp://www.kmis.ic3k.org\t2013-09-19\t2013-09-22\t\t2013-05-16\t2013-06-24\t2013-07-05\t32\t12']

In [37]:
temp_var_ci = txt_file_ci.map(lambda k : k.split("\t"))

In [38]:
df_ci=temp_var_ci.toDF()

In [39]:
df_ci.take(5)

[Row(_1=u'2312404054', _2=u'25000', _3=u'siggraph 2016', _4=u'SIGGRAPH 2016', _5=u'1164321581', _6=u'Anaheim, California', _7=u'http://s2016.siggraph.org/call-submissions', _8=u'2016-07-24', _9=u'2016-07-28', _10=u'', _11=u'2016-01-18', _12=u'', _13=u'2016-01-19', _14=u'846', _15=u'1741'),
 Row(_1=u'121793801', _2=u'25000', _3=u'wsdm 2012', _4=u'WSDM 2012', _5=u'1120384002', _6=u'Seattle, WA', _7=u'http://wsdm2012.org/', _8=u'2012-02-08', _9=u'2012-02-12', _10=u'2011-08-04', _11=u'2011-08-11', _12=u'2011-10-26', _13=u'', _14=u'87', _15=u'3496'),
 Row(_1=u'2625789444', _2=u'25000', _3=u'iprm 2006', _4=u'IPRM 2006', _5=u'2621445615', _6=u'Princeton, NJ, USA', _7=u'', _8=u'', _9=u'', _10=u'', _11=u'', _12=u'', _13=u'', _14=u'', _15=u''),
 Row(_1=u'82574969', _2=u'25000', _3=u'foga 2009', _4=u'FOGA 2009', _5=u'1159491267', _6=u'Orlando, FL, USA', _7=u'http://www.sigevo.org/foga-2009', _8=u'2009-01-09', _9=u'2009-01-11', _10=u'', _11=u'2008-09-01', _12=u'', _13=u'', _14=u'19', _15=u'382'),


In [40]:
df_ci.show(5)

+----------+-----+-------------+-------------+----------+--------------------+--------------------+----------+----------+----------+----------+----------+----------+---+----+
|        _1|   _2|           _3|           _4|        _5|                  _6|                  _7|        _8|        _9|       _10|       _11|       _12|       _13|_14| _15|
+----------+-----+-------------+-------------+----------+--------------------+--------------------+----------+----------+----------+----------+----------+----------+---+----+
|2312404054|25000|siggraph 2016|SIGGRAPH 2016|1164321581| Anaheim, California|http://s2016.sigg...|2016-07-24|2016-07-28|          |2016-01-18|          |2016-01-19|846|1741|
| 121793801|25000|    wsdm 2012|    WSDM 2012|1120384002|         Seattle, WA|http://wsdm2012.org/|2012-02-08|2012-02-12|2011-08-04|2011-08-11|2011-10-26|          | 87|3496|
|2625789444|25000|    iprm 2006|    IPRM 2006|2621445615|  Princeton, NJ, USA|                    |          |          |    

In [41]:
#df_ci.printSchema()#shows that all are type of string and nullable=true

In [42]:
df_ci = df_ci.withColumn("ConfInstanceID",df_ci["_1"].cast(LongType()))

In [43]:
df_ci = df_ci.withColumn("CIRank",df_ci["_2"].cast(IntegerType()))

In [44]:
df_ci = df_ci.withColumn("CINormalizedNm",df_ci["_3"].cast(StringType()))

In [45]:
df_ci = df_ci.withColumn("CIDisplayNm",df_ci["_4"].cast(StringType()))

In [46]:
df_ci = df_ci.withColumn("CICSID",df_ci["_5"].cast(LongType()))

In [47]:
df_ci = df_ci.withColumn("Location",df_ci["_6"].cast(StringType()))

In [48]:
df_ci = df_ci.withColumn("CIOfficialURL",df_ci["_7"].cast(StringType()))

In [49]:
df_ci = df_ci.withColumn("StartDate",df_ci["_8"].cast(DateType()))

In [50]:
df_ci = df_ci.withColumn("EndDate",df_ci["_9"].cast(DateType()))

In [51]:
df_ci = df_ci.withColumn("AbstractRegDate",df_ci["_10"].cast(DateType()))

In [52]:
df_ci = df_ci.withColumn("SubDeadlineDate",df_ci["_11"].cast(DateType()))

In [53]:
df_ci = df_ci.withColumn("NotificationDueDate",df_ci["_12"].cast(DateType()))

In [54]:
df_ci = df_ci.withColumn("FinalVersionDueDate",df_ci["_13"].cast(DateType()))

In [55]:
df_ci = df_ci.withColumn("CIPaperCount",df_ci["_14"].cast(IntegerType()))

In [56]:
df_ci = df_ci.withColumn("CIPaperCitationCnt",df_ci["_15"].cast(IntegerType()))

In [57]:
df_ci.show(2)

+----------+-----+-------------+-------------+----------+-------------------+--------------------+----------+----------+----------+----------+----------+----------+---+----+--------------+------+--------------+-------------+----------+-------------------+--------------------+----------+----------+---------------+---------------+-------------------+-------------------+------------+------------------+
|        _1|   _2|           _3|           _4|        _5|                 _6|                  _7|        _8|        _9|       _10|       _11|       _12|       _13|_14| _15|ConfInstanceID|CIRank|CINormalizedNm|  CIDisplayNm|    CICSID|           Location|       CIOfficialURL| StartDate|   EndDate|AbstractRegDate|SubDeadlineDate|NotificationDueDate|FinalVersionDueDate|CIPaperCount|CIPaperCitationCnt|
+----------+-----+-------------+-------------+----------+-------------------+--------------------+----------+----------+----------+----------+----------+----------+---+----+--------------+------

In [58]:
drop_columns =["_1","_2","_3","_4","_5","_6","_7","_8","_9","_10","_11","_12","_13","_14","_15"]
df_ci = df_ci.drop(*drop_columns)    

In [59]:
df_ci.show(2)

+--------------+------+--------------+-------------+----------+-------------------+--------------------+----------+----------+---------------+---------------+-------------------+-------------------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|  CIDisplayNm|    CICSID|           Location|       CIOfficialURL| StartDate|   EndDate|AbstractRegDate|SubDeadlineDate|NotificationDueDate|FinalVersionDueDate|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-------------+----------+-------------------+--------------------+----------+----------+---------------+---------------+-------------------+-------------------+------------+------------------+
|    2312404054| 25000| siggraph 2016|SIGGRAPH 2016|1164321581|Anaheim, California|http://s2016.sigg...|2016-07-24|2016-07-28|           null|     2016-01-18|               null|         2016-01-19|         846|              1741|
|     121793801| 25000|     wsdm 2012|    WSDM 2012|1120384002|        Seatt

In [60]:
df_ci.describe().show()

+-------+--------------------+-------+--------------+-------------+--------------------+--------------+--------------------+------------------+------------------+
|summary|      ConfInstanceID| CIRank|CINormalizedNm|  CIDisplayNm|              CICSID|      Location|       CIOfficialURL|      CIPaperCount|CIPaperCitationCnt|
+-------+--------------------+-------+--------------+-------------+--------------------+--------------+--------------------+------------------+------------------+
|  count|               13632|  13632|         13632|        13632|               13632|         13632|               13632|              9958|              9958|
|   mean| 1.250923939761517E9|25000.0|          null|         null|1.8119981032470658E9|          null|                null|125.96153846153847| 862.6626832697328|
| stddev|1.1853717357293985E9|    0.0|          null|         null| 7.435169239046494E8|          null|                null|224.74227934025276| 2360.415811174456|
|    min|             

In [61]:
df_ci.select([count(when(col(c).isNull(),c)).alias(c) for c in df_ci.columns]).show()

+--------------+------+--------------+-----------+------+--------+-------------+---------+-------+---------------+---------------+-------------------+-------------------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|CIDisplayNm|CICSID|Location|CIOfficialURL|StartDate|EndDate|AbstractRegDate|SubDeadlineDate|NotificationDueDate|FinalVersionDueDate|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-----------+------+--------+-------------+---------+-------+---------------+---------------+-------------------+-------------------+------------+------------------+
|             0|     0|             0|          0|     0|       0|            0|     2549|   2548|          11754|           4471|               7607|               8577|        3674|              3674|
+--------------+------+--------------+-----------+------+--------+-------------+---------+-------+---------------+---------------+-------------------+-------------------+------------+-----

In [62]:
df_ci.filter((df_ci["StartDate"] == "") | df_ci["StartDate"].isNull()).count()

2549

In [63]:
df_ci.filter((df_ci["CIPaperCitationCnt"] == "") | df_ci["CIPaperCitationCnt"].isNull()).count()

3674

In [64]:
df_ci.filter((df_ci["NotificationDueDate"] == "") | df_ci["NotificationDueDate"].isNull()).count()

7607